# CHECKING REGRESSION OF SINGLE ELEMENT

It is required to check how different will look model training in case of single variable respective to the double variables training

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os.path
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers

import gc

from point_net_api import *

tf.random.set_seed(1234)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import sklearn.metrics as sm

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6329328866984323034
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5750390784
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14535464429262148754
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
1 Physical GPUs, 1 Logical GPUs
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6



## Reading the data, forming TF slices

In [2]:
train_df = read_all_pickles(package_name='pkls_valence_arousal', file_specification="_landmarks_valence_arousal.pkl")
print('original length = ' + str(len(train_df)))

#   drop empty records
train_df.dropna(inplace=True)

#   ignoring the last three labels because they are not providing necessary information
train_df = train_df[(train_df.emotion != 8) & (train_df.emotion != 9) & (train_df.emotion != 10)]
train_df.drop(columns=['emotion'], inplace=True)

print('updated length = ' + str(len(train_df)))
train_df.head(10)

original length = 409797
updated length = 272340


,path,valence,arousal,landmarks
400002,Manually_Annotated_Images/325/f489e8169ea30897...,0.577985,0.271739,"[[0.5394599437713623, 0.676323413848877, -0.08..."
400003,Manually_Annotated_Images/264/224ee36d52b497b1...,-0.384534,0.110619,"[[0.5218021273612976, 0.7440579533576965, -0.1..."
400005,Manually_Annotated_Images/980/8262220f6a3bcfd7...,0.639259,0.034451,"[[0.5117629766464233, 0.7039216160774231, -0.0..."
400007,Manually_Annotated_Images/609/5b3d3b8b9d89deaf...,0.705858,-0.258112,"[[0.47549158334732056, 0.7176483273506165, -0...."
400009,Manually_Annotated_Images/958/7b235cdca2ca0430...,-0.103175,-0.341270,"[[0.5392422080039978, 0.7580064535140991, -0.0..."
400013,Manually_Annotated_Images/715/bff238e798b70ccf...,-0.595238,0.428571,"[[0.49899062514305115, 0.7014129161834717, -0...."
400014,Manually_Annotated_Images/514/f92fad99d07e4b9e...,-0.101626,0.895277,"[[0.47227370738983154, 0.6721517443656921, -0...."
400015,Manually_Annotated_Images/728/9629f4764d5956d9...,0.526225,-0.081953,"[[0.519411563873291, 0.739055871963501, -0.060..."
400016,Manually_Annotated_Images/866/ec55abeb8a12d104...,0.871080,0.183895,"[[0.5619162917137146, 0.6456629633903503, -0.0..."
400017,Manually_Annotated_Images/907/7afaf57f140a6270...,0.691384,-0.062657,"[[0.5264552235603333, 0.7333130240440369, -0.0..."


In [3]:
X = np.array(list(map(np.float64, np.array(train_df['landmarks']))))
Y_valence = np.array(train_df['valence'])

x_train, x_test, y_train_valence, y_test_valence = train_test_split(X, Y_valence, test_size=0.2, random_state=42)

In [4]:
#   to evade memory overflow it is required to remove unnecessary data
del X
del Y_valence
del train_df
gc.collect()

22

In [5]:
train_valence_tf_df = tf.data.Dataset.from_tensor_slices((x_train, y_train_valence))
test_valence_tf_df = tf.data.Dataset.from_tensor_slices((x_test, y_test_valence))

train_valence_tf_df = train_valence_tf_df.shuffle(len(x_train)).batch(32)
test_valence_tf_df = test_valence_tf_df.shuffle(len(x_train)).batch(32)

In [6]:
del y_train_valence
del y_test_valence
gc.collect()

del x_train
del x_test
gc.collect()

0

# Version 2.3 - checking leaky RELU for activation, setting kernel size to 3, MSE

## Forming NN

In [15]:
first_degree, second_degree, third_degree, fourth_degree = 128, 256, 512, 1024

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree, 'leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu', kernel_size=3)
x = conv_bn(x, 64, activation_function='leaky_relu', kernel_size=3)
x = tnet(x, 64, first_degree, second_degree, third_degree, fourth_degree, activation_function='leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu', kernel_size=3)
x = conv_bn(x, first_degree, activation_function='leaky_relu', kernel_size=3)
x = conv_bn(x, third_degree, activation_function='leaky_relu', kernel_size=3)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree, activation_function='leaky_relu')
x = layers.Dropout(0.25)(x)
x = dense_bn(x, first_degree, activation_function='leaky_relu')
x = layers.Dropout(0.25)(x)

outputs = layers.Dense(1, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d_11 (Conv1D)             (None, 468, 32)      128         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_17 (BatchN  (None, 468, 32)     128         ['conv1d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 leaky_re_lu_17 (LeakyReLU)     (None, 468, 32)      0           ['batch_normalization

## Training NN

In [16]:
model.compile(loss="mean_squared_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            metrics=["mse", 'mae'],)

model.fit(train_valence_tf_df, epochs=8, validation_data=test_valence_tf_df)

Epoch 1/8
6809/6809 [==============================] - 360s 50ms/step - loss: 2.3430 - mse: 0.2349 - mae: 0.3751 - val_loss: 2.2550 - val_mse: 0.1695 - val_mae: 0.3167
Epoch 2/8
6809/6809 [==============================] - 346s 51ms/step - loss: 2.2404 - mse: 0.1677 - mae: 0.3142 - val_loss: 2.2537 - val_mse: 0.1812 - val_mae: 0.3110
Epoch 3/8
6809/6809 [==============================] - 378s 55ms/step - loss: 2.2348 - mse: 0.1626 - mae: 0.3087 - val_loss: 2.2309 - val_mse: 0.1587 - val_mae: 0.2945
Epoch 4/8
6809/6809 [==============================] - 354s 52ms/step - loss: 2.2314 - mse: 0.1592 - mae: 0.3048 - val_loss: 2.2323 - val_mse: 0.1599 - val_mae: 0.3081
Epoch 5/8
6809/6809 [==============================] - 353s 52ms/step - loss: 2.2284 - mse: 0.1562 - mae: 0.3013 - val_loss: 2.2229 - val_mse: 0.1507 - val_mae: 0.2979
Epoch 6/8
6809/6809 [==============================] - 354s 52ms/step - loss: 2.2256 - mse: 0.1534 - mae: 0.2981 - val_loss: 2.2232 - val_mse: 0.1511 - val_mae:

## Validating NN

In [17]:
true_valences, pred_valences = [], []

for batch in test_valence_tf_df.as_numpy_iterator():
  batch_predictions = model.predict(batch[0])
  for index in range(len(batch_predictions)):
    true_valences.append(batch[1][index])
    pred_valences.append(batch_predictions[index])
print('collected data for metrics')

1/1 [==============================] - 1s 759ms/step
collected data for metrics


In [18]:
print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))

Mean absolute error = 0.3
Mean squared error = 0.17
Median absolute error = 0.21
Explain variance score = 0.42
R2 score = 0.38


# Version 2.4 - checking standard RELU as activation with Convolution of kernel size 5, MSE

## Forming NN

In [24]:
first_degree, second_degree, third_degree, fourth_degree = 128, 256, 512, 1024

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 64, kernel_size=5)
x = conv_bn(x, 64, kernel_size=5)
x = tnet(x, 64, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 64, kernel_size=5)
x = conv_bn(x, first_degree, kernel_size=5)
x = conv_bn(x, third_degree, kernel_size=5)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, first_degree)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(1, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d_44 (Conv1D)             (None, 468, 32)      128         ['input_5[0][0]']                
                                                                                                  
 batch_normalization_60 (BatchN  (None, 468, 32)     128         ['conv1d_44[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_23 (Activation)     (None, 468, 32)      0           ['batch_normalization

## Training NN

In [25]:
model.compile(loss="mean_squared_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.002),
            metrics=["mse", 'mae'],)

model.fit(train_valence_tf_df, epochs=8, validation_data=test_valence_tf_df)

Epoch 1/8
6809/6809 [==============================] - 336s 48ms/step - loss: 15753.9492 - mse: 0.4475 - mae: 0.4663 - val_loss: 5.6386 - val_mse: 0.2825 - val_mae: 0.4218
Epoch 2/8
6809/6809 [==============================] - 325s 48ms/step - loss: 2.3866 - mse: 0.2205 - mae: 0.3797 - val_loss: 2.2845 - val_mse: 0.1967 - val_mae: 0.3497
Epoch 3/8
6809/6809 [==============================] - 323s 47ms/step - loss: 2.2877 - mse: 0.1870 - mae: 0.3380 - val_loss: 67.8440 - val_mse: 0.9475 - val_mae: 0.4859
Epoch 4/8
6809/6809 [==============================] - 325s 48ms/step - loss: 31077.9707 - mse: 0.2602 - mae: 0.4431 - val_loss: 45.9585 - val_mse: 0.2645 - val_mae: 0.4416
Epoch 5/8
6809/6809 [==============================] - 335s 49ms/step - loss: 37.3370 - mse: 0.2633 - mae: 0.4483 - val_loss: 3.2524 - val_mse: 0.2657 - val_mae: 0.4507
Epoch 6/8
6809/6809 [==============================] - 328s 48ms/step - loss: 5.0178 - mse: 0.2633 - mae: 0.4488 - val_loss: 2.4053 - val_mse: 0.2659

## Validating NN

In [26]:
true_valences, pred_valences = [], []

for batch in test_valence_tf_df.as_numpy_iterator():
  batch_predictions = model.predict(batch[0])
  for index in range(len(batch_predictions)):
    true_valences.append(batch[1][index])
    pred_valences.append(batch_predictions[index])
print('collected data for metrics')

1/1 [==============================] - 1s 600ms/step
collected data for metrics


In [27]:
print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))

Mean absolute error = 0.32
Mean squared error = 0.17
Median absolute error = 0.25
Explain variance score = 0.38
R2 score = 0.37


# Version 2.5 - checking RELU with convolution kernel size 10, MSE

In [28]:
first_degree, second_degree, third_degree, fourth_degree = 128, 256, 512, 1024

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 64, kernel_size=10)
x = conv_bn(x, 64, kernel_size=10)
x = tnet(x, 64, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 64, kernel_size=10)
x = conv_bn(x, first_degree, kernel_size=10)
x = conv_bn(x, third_degree, kernel_size=10)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree)
x = layers.Dropout(0.25)(x)
x = dense_bn(x, first_degree)
x = layers.Dropout(0.25)(x)

outputs = layers.Dense(1, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d_55 (Conv1D)             (None, 468, 32)      128         ['input_6[0][0]']                
                                                                                                  
 batch_normalization_71 (BatchN  (None, 468, 32)     128         ['conv1d_55[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_40 (Activation)     (None, 468, 32)      0           ['batch_normalization

In [29]:
model.compile(loss="mean_squared_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.002),
            metrics=["mse", 'mae'],)

model.fit(train_valence_tf_df, epochs=8, validation_data=test_valence_tf_df)

Epoch 1/8
6809/6809 [==============================] - 358s 51ms/step - loss: 17143.2832 - mse: 0.4528 - mae: 0.4845 - val_loss: 2.9882 - val_mse: 0.3775 - val_mae: 0.4572
Epoch 2/8
6809/6809 [==============================] - 348s 51ms/step - loss: 2.4256 - mse: 0.2412 - mae: 0.4089 - val_loss: 2.5510 - val_mse: 0.2093 - val_mae: 0.3454
Epoch 3/8
6809/6809 [==============================] - 349s 51ms/step - loss: 16782.0078 - mse: 0.1960 - mae: 0.3506 - val_loss: 2011.4545 - val_mse: 0.2808 - val_mae: 0.4556
Epoch 4/8
6809/6809 [==============================] - 349s 51ms/step - loss: 53.3417 - mse: 0.2497 - mae: 0.4241 - val_loss: 117.2709 - val_mse: 0.4667 - val_mae: 0.4542
Epoch 5/8
6809/6809 [==============================] - 350s 51ms/step - loss: 11.8750 - mse: 0.2511 - mae: 0.4257 - val_loss: 2.4757 - val_mse: 0.2502 - val_mae: 0.4234
Epoch 6/8
6809/6809 [==============================] - 349s 51ms/step - loss: 75.9218 - mse: 0.2473 - mae: 0.4196 - val_loss: 233.1905 - val_mse:

In [30]:
true_valences, pred_valences = [], []

for batch in test_valence_tf_df.as_numpy_iterator():
  batch_predictions = model.predict(batch[0])
  for index in range(len(batch_predictions)):
    true_valences.append(batch[1][index])
    pred_valences.append(batch_predictions[index])
print('collected data for metrics')

print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))

1/1 [==============================] - 1s 612ms/step
collected data for metrics
Mean absolute error = 0.33
Mean squared error = 0.19
Median absolute error = 0.25
Explain variance score = 0.32
R2 score = 0.31


# Version 2.6 - checking RELU with convolution kernel size 1, even deeper network, MSE

In [33]:
first_degree, second_degree, third_degree, fourth_degree = 256, 512, 1024, 2048

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree, activation_function='leaky_relu')
x = conv_bn(x, 128, activation_function='leaky_relu')
x = conv_bn(x, 128, activation_function='leaky_relu')
x = tnet(x, 128, first_degree, second_degree, third_degree, fourth_degree, activation_function='leaky_relu')
x = conv_bn(x, 128, activation_function='leaky_relu')
x = conv_bn(x, first_degree, activation_function='leaky_relu')
x = conv_bn(x, third_degree, activation_function='leaky_relu')
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree)
x = layers.Dropout(0.25)(x)
x = dense_bn(x, first_degree)
x = layers.Dropout(0.25)(x)

outputs = layers.Dense(1, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d_77 (Conv1D)             (None, 468, 32)      128         ['input_8[0][0]']                
                                                                                                  
 batch_normalization_93 (BatchN  (None, 468, 32)     128         ['conv1d_77[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 leaky_re_lu_45 (LeakyReLU)     (None, 468, 32)      0           ['batch_normalization

In [34]:
model.compile(loss="mean_squared_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            metrics=["mse", 'mae'],)

model.fit(train_valence_tf_df, epochs=10, validation_data=test_valence_tf_df)

Epoch 1/10
6809/6809 [==============================] - 582s 84ms/step - loss: 28.9584 - mse: 0.4240 - mae: 0.4614 - val_loss: 334.9045 - val_mse: 108.4505 - val_mae: 2.6739
Epoch 2/10
6809/6809 [==============================] - 571s 84ms/step - loss: 4.7449 - mse: 0.2934 - mae: 0.4624 - val_loss: 868.5998 - val_mse: 286.0815 - val_mae: 3.5436
Epoch 3/10
6809/6809 [==============================] - 571s 84ms/step - loss: 39.1260 - mse: 0.2388 - mae: 0.4020 - val_loss: 4.4472 - val_mse: 0.2814 - val_mae: 0.4547
Epoch 4/10
6809/6809 [==============================] - 572s 84ms/step - loss: 4.4061 - mse: 0.2430 - mae: 0.4158 - val_loss: 4.3253 - val_mse: 0.2183 - val_mae: 0.3800
Epoch 5/10
6809/6809 [==============================] - 572s 84ms/step - loss: 4.3214 - mse: 0.1806 - mae: 0.3291 - val_loss: 13.0617 - val_mse: 0.2275 - val_mae: 0.3663
Epoch 6/10
6809/6809 [==============================] - 572s 84ms/step - loss: 4.2998 - mse: 0.1725 - mae: 0.3192 - val_loss: 4.6786 - val_mse: 

In [35]:
true_valences, pred_valences = [], []

for batch in test_valence_tf_df.as_numpy_iterator():
  batch_predictions = model.predict(batch[0])
  for index in range(len(batch_predictions)):
    true_valences.append(batch[1][index])
    pred_valences.append(batch_predictions[index])
print('collected data for metrics')

print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))

1/1 [==============================] - 1s 711ms/step
collected data for metrics
Mean absolute error = 0.3
Mean squared error = 0.16
Median absolute error = 0.22
Explain variance score = 0.42
R2 score = 0.42


# Version 2.7 - checking MAE loss with very deep NN

In [7]:
first_degree, second_degree, third_degree, fourth_degree = 256, 512, 1024, 2048

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree, activation_function='leaky_relu')
x = conv_bn(x, 128, activation_function='leaky_relu')
x = conv_bn(x, 128, activation_function='leaky_relu')
x = tnet(x, 128, first_degree, second_degree, third_degree, fourth_degree, activation_function='leaky_relu')
x = conv_bn(x, 128, activation_function='leaky_relu')
x = conv_bn(x, first_degree, activation_function='leaky_relu')
x = conv_bn(x, third_degree, activation_function='leaky_relu')
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree)
x = layers.Dropout(0.25)(x)
x = dense_bn(x, first_degree)
x = layers.Dropout(0.25)(x)

outputs = layers.Dense(1, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 468, 32)      128         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 468, 32)     128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 468, 32)      0           ['batch_normalization

In [8]:
model.compile(loss="mean_absolute_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.002),
            metrics=["mse", 'mae'],)

model.fit(train_valence_tf_df, epochs=10, validation_data=test_valence_tf_df)

Epoch 1/10
6809/6809 [==============================] - 565s 81ms/step - loss: 5032.7876 - mse: 0.6986 - mae: 0.5270 - val_loss: 63.2307 - val_mse: 3.9351 - val_mae: 0.8277
Epoch 2/10
6809/6809 [==============================] - 558s 82ms/step - loss: 5.0611 - mse: 0.2878 - mae: 0.4533 - val_loss: 4.6509 - val_mse: 0.2934 - val_mae: 0.4484
Epoch 3/10
6809/6809 [==============================] - 557s 82ms/step - loss: 12356.3301 - mse: 0.2725 - mae: 0.4366 - val_loss: 147.1322 - val_mse: 0.2936 - val_mae: 0.4554
Epoch 4/10
6809/6809 [==============================] - 558s 82ms/step - loss: 30.2143 - mse: 0.2729 - mae: 0.4475 - val_loss: 5.6145 - val_mse: 0.2865 - val_mae: 0.4473
Epoch 5/10
6809/6809 [==============================] - 559s 82ms/step - loss: 4.8330 - mse: 0.2569 - mae: 0.4131 - val_loss: 4.6308 - val_mse: 0.2745 - val_mae: 0.4097
Epoch 6/10
6809/6809 [==============================] - 560s 82ms/step - loss: 1232.7721 - mse: 0.2329 - mae: 0.3861 - val_loss: 17.1696 - val_m

KeyboardInterrupt: 

In [ ]:
true_valences, pred_valences = [], []

for batch in test_valence_tf_df.as_numpy_iterator():
  batch_predictions = model.predict(batch[0])
  for index in range(len(batch_predictions)):
    true_valences.append(batch[1][index])
    pred_valences.append(batch_predictions[index])
print('collected data for metrics')

print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))